In [2]:
import pickle
import time
import os
import numpy as np
import sys
import music21
from keras.models import load_model
import matplotlib.pyplot as plt
from own_functions import create_network, sample_with_temp


In [7]:
distincts = pickle.load(open("../data/stored_data/parsed_data/distincts", "rb"))
note_names, n_notes, duration_names, n_durations = distincts

lookups = pickle.load(open("../data/stored_data/parsed_data/lookups", "rb"))
note_to_int, int_to_note, duration_to_int, int_to_duration = lookups

In [8]:
model = load_model("../data/stored_data/models/bach_all.h5")

In [9]:
# prediction params
notes_temp=0.5
duration_temp = 0.5
max_extra_notes = 50
max_seq_len = 32
seq_len = 32

#random start: 
#notes = ['START', 'F#3', 'G#3', 'F#3', 'E3', 'F#3', 'G#3', 'F#3', 'E3', 'F#3', 'G#3', 'F#3', 'E3','F#3', 'G#3', 'F#3', 'E3', 'F#3', 'G#3', 'F#3', 'E3', 'F#3', 'G#3', 'F#3', 'E3']
#durations = [0, 0.75, 0.25, 1, 1, 1, 2, 0.75, 0.25, 1, 1, 1, 2, 0.75, 0.25, 1, 1, 1, 2, 0.75, 0.25, 1, 1, 1, 2]


#happy-birthday start:
#notes = ['START', 'D3', 'D3', 'E3', 'D3', 'G3', 'F#3','D3', 'D3', 'E3', 'D3', 'A3', 'G3','D3', 'D3', 'E3', 'D3', 'G3', 'F#3','D3', 'D3', 'E3', 'D3', 'A3', 'G3']
#durations = [0, 0.75, 0.25, 1, 1, 1, 2, 0.75, 0.25, 1, 1, 1, 2, 0.75, 0.25, 1, 1, 1, 2, 0.75, 0.25, 1, 1, 1, 2]


#christmas-song start: 
#notes = ['START',"D3", "G3", "G3", "A3", "G3", "F#3", "E3", "E3", "E3", "A3", "A3", "B3", "A3", "G3", "F#3", "D3", "D3", "B3", "B3", "C4", "B3", "A3", "G3", "E3"]
#durations = [0, 1, 1, 0.5,0.5,0.5,0.5,1,1,1,1,0.5,0.5,0.5,0.5,1,1,1,1,0.5,0.5,0.5,0.5,1,1]

notes = ['START']
durations = [0]

# adding no-notes ('START') until the sequence is of 32 notes
if seq_len is not None:
    notes = ['START'] * (seq_len - len(notes)) + notes
    durations = [0] * (seq_len - len(durations)) + durations

sequence_length = len(notes)

In [13]:
prediction_output = []
notes_input_sequence = []
durations_input_sequence = []

overall_preds = []

# parsing starting sequence: 
for n, d in zip(notes,durations):
    note_int = note_to_int[n]
    duration_int = duration_to_int[d]
    
    notes_input_sequence.append(note_int)
    durations_input_sequence.append(duration_int)
    
    prediction_output.append([n, d])
    
    if n != 'START':
        midi_note = music21.note.Note(n)

        new_note = np.zeros(128)
        new_note[midi_note.pitch.midi] = 1
        overall_preds.append(new_note)
        
# predicting :

for note_index in range(max_extra_notes):

    prediction_input = [np.array([notes_input_sequence]), np.array([durations_input_sequence])]

    notes_prediction, durations_prediction = model.predict(prediction_input, verbose=0)
    
    new_note = np.zeros(128)
    
    for idx, n_i in enumerate(notes_prediction[0]):
        try:
            note_name = int_to_note[idx]
            midi_note = music21.note.Note(note_name)
            new_note[midi_note.pitch.midi] = n_i
            
        except:
            pass
        
    overall_preds.append(new_note)
            
    
    i1 = sample_with_temp(notes_prediction[0], notes_temp)
    i2 = sample_with_temp(durations_prediction[0], duration_temp)
    

    note_result = int_to_note[i1]
    duration_result = int_to_duration[i2]
    
    prediction_output.append([note_result, duration_result])

    notes_input_sequence.append(i1)
    durations_input_sequence.append(i2)
    
    if len(notes_input_sequence) > max_seq_len:
        notes_input_sequence = notes_input_sequence[1:]
        durations_input_sequence = durations_input_sequence[1:]
        
#     print(note_result)
#     print(duration_result)
        
    if note_result == 'START':
        break

overall_preds = np.transpose(np.array(overall_preds)) 
print('Generated sequence of '+str(len(prediction_output))+' notes')

Generated sequence of 82 notes


In [388]:


midi_stream = music21.stream.Stream()

# create note and chord objects based on the values generated by the model
for pattern in prediction_output:
    note_pattern, duration_pattern = pattern
    # pattern is a chord
    if ('.' in note_pattern):
        notes_in_chord = note_pattern.split('.')
        chord_notes = []
        for current_note in notes_in_chord:
            new_note = music21.note.Note(current_note)
            new_note.duration = music21.duration.Duration(duration_pattern)
            new_note.storedInstrument = music21.instrument.Piano()
            chord_notes.append(new_note)
        new_chord = music21.chord.Chord(chord_notes)
        midi_stream.append(new_chord)
    elif note_pattern == 'rest':
    # pattern is a rest
        new_note = note.Rest()
        new_note.duration = duration.Duration(duration_pattern)
        new_note.storedInstrument = instrument.Piano()
        midi_stream.append(new_note)
    elif note_pattern != 'START':
    # pattern is a note
        new_note = music21.note.Note(note_pattern)
        new_note.duration = music21.duration.Duration(duration_pattern)
        new_note.storedInstrument = music21.instrument.Piano()
        midi_stream.append(new_note)



midi_stream = midi_stream.chordify()
timestr = time.strftime("%Y%m%d-%H%M%S")
midi_stream.write('midi', fp=os.path.join("../data/generated_songs/", 'output-song-' + timestr + '.mid'))

'../data/generated_songs/output-song-20201217-135401.mid'